In [2]:
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model

# Load test data
test = pd.read_csv("test.csv", sep=r'\s+', header=None)
test.columns = ['unit', 'time'] + [f'op_setting_{i}' for i in range(1, 4)] + \
               [f'sensor_{i}' for i in range(1, 22)]

# Load pre-trained objects
model = load_model("gru_rul_model.h5")
scaler = joblib.load("scaler.pkl")
pca = joblib.load("pca.pkl")

# Parameters
SEQUENCE_LENGTH = 30
MAX_RUL = 130

# Select same features as training
selected_features = scaler.feature_names_in_

# Normalize
test[selected_features] = scaler.transform(test[selected_features])

# Apply PCA
test_pca = pca.transform(test[selected_features])
test_pca_df = pd.DataFrame(test_pca)
test_pca_df['unit'] = test['unit'].values
test_pca_df['time'] = test['time'].values

# Sequence generation for last window of each engine
def last_sequence(df, seq_len):
    sequences = []
    units = []
    for unit in df['unit'].unique():
        unit_df = df[df['unit'] == unit].sort_values('time')
        if len(unit_df) >= seq_len:
            seq = unit_df.iloc[-seq_len:][df.columns.difference(['unit', 'time'])].values
        else:
            # Pad sequence with zeros if shorter than SEQUENCE_LENGTH
            pad = np.zeros((seq_len - len(unit_df), unit_df.shape[1] - 2))
            seq = np.vstack((pad, unit_df[df.columns.difference(['unit', 'time'])].values))
        sequences.append(seq)
        units.append(unit)
    return np.array(sequences), units

X_test, units = last_sequence(test_pca_df, SEQUENCE_LENGTH)

# Predict RUL
y_pred = model.predict(X_test).flatten()

# Clip predicted RUL to same max as training
y_pred = np.clip(y_pred, 0, MAX_RUL)

# Output DataFrame
rul_pred_df = pd.DataFrame({'unit': units, 'Predicted_RUL': y_pred})
print(rul_pred_df)

# Optionally save to CSV
rul_pred_df.to_csv("predicted_rul_test.csv", index=False)


2025-05-04 18:18:51.486573: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


7/7 [==============================] - 0s 3ms/step
     unit  Predicted_RUL
0       1     110.536179
1       2      48.303345
2       3      79.615524
3       4      63.776028
4       5      96.192085
..    ...            ...
213   214      20.831123
214   215     128.791275
215   216      86.251068
216   217     112.444801
217   218     116.346313

[218 rows x 2 columns]
